# In this notebook the function is calculates L was tested and incrementally sped up with numpy

In [2]:
import numpy as np
import os
import pickle
from numba import jit
import math
import cmath

import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


Initializing a test function, a Gaussian in this case.

In [4]:
def gauss(x, y, x0=0, y0=0):
    return 1/np.sqrt(np.pi) * np.exp(-0.5*((x-x0)**2+(y-y0)**2))

In [5]:
X = 101
Y = 101
a = 10
x = np.linspace(-a,a, X)
y = np.linspace(-a,a, Y)

In [6]:
xx, yy = np.meshgrid(x, y, indexing='ij')
psi = gauss(xx, yy) + (0+0j)
plt.imshow(np.abs(psi))
plt.colorbar()
plt.show()

### Step by step trying to understand how to speed up the calculation of L with numpy

Mark I: A very literal copy of the math presented in the paper

In [7]:
@jit
def L_MK1(psi, psi_hat, a, X, Y):
    psi_dx = np.ones((X,Y)) + (0+0j)
    psi_dy = np.ones((X,Y)) + (0+0j)
    psi_L = np.ones((X,Y)) + (0+0j)
    
    for j in range(X):
        for k in range(Y):
            s_dx = 0
            s_dy = 0
            for p in range(-X//2, X//2):
                for q in range(-Y//2, Y//2):
                    my_p = 2*p*math.pi/(2*a)
                    lambda_q = 2*q*math.pi/(2*a)
                    tmp = psi_hat[p, q] * cmath.exp(1j*2*j*p*np.pi/X) * cmath.exp(1j*2*k*q*np.pi/Y)
                    s_dx += my_p * tmp
                    s_dy += lambda_q * tmp
            psi_dx[j,k] = s_dx / (X**2)
            psi_dy[j,k] = s_dy / (Y**2)
    
    dx = 2*a/X
    dy = 2*a/Y
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dy)
    
    for j in range(X):
        for k in range(Y):
            psi_L[j,k] = x[j]*psi_dy[j,k] - y[k]*psi_dx[j,k]
    return psi_L

psi_hat = np.fft.fft2(psi)#/N**2
l = L_MK1(psi, psi_hat, a, X, Y)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

Mark II: replacing the last double for loop by numpy magic

In [8]:
@jit
def L_MK2(psi, psi_hat, a, X, Y):
    psi_dx = np.ones((X,Y)) + (0+0j)
    psi_dy = np.ones((X,Y)) + (0+0j)
    psi_L = np.ones((X,Y)) + (0+0j)
    
    for j in range(X):
        for k in range(Y):
            s_dx = 0
            s_dy = 0
            for p in range(-X//2, X//2):
                for q in range(-Y//2, Y//2):
                    my_p = 2*p*math.pi/(2*a)
                    lambda_q = 2*q*math.pi/(2*a)
                    tmp = psi_hat[p, q] * cmath.exp(1j*2*j*p*np.pi/X) * cmath.exp(1j*2*k*q*np.pi/Y)
                    s_dx += my_p * tmp
                    s_dy += lambda_q * tmp
            psi_dx[j,k] = s_dx / (X**2)
            psi_dy[j,k] = s_dy / (Y**2)
    
    dx = 2*a/X
    dy = 2*a/Y
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dy)
    
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='ij')
    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L

psi_hat = np.fft.fft2(psi)#/N**2
l = L_MK2(psi, psi_hat, a, X, Y)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

Mark III: Replacing the direct fourier transform by the fast fourier transform

In [9]:
@jit
def L_MK3(psi, psi_hat, a, X, Y):
    psi_dx = np.ones((X,Y)) + (0+0j)
    psi_dy = np.ones((X,Y)) + (0+0j)
    psi_dx_hat = np.ones((X,Y)) + (0+0j)
    psi_dy_hat = np.ones((X,Y)) + (0+0j)
    psi_L = np.ones((X,Y)) + (0+0j)
    
    
    for p in range(-X//2, X//2):
        for q in range(-Y//2, Y//2):
            my_p = 2*p*math.pi/(2*a)
            lambda_q = 2*q*math.pi/(2*a)
            psi_dx_hat[p,q] = psi_hat[p, q] * my_p
            psi_dy_hat[p,q] = psi_hat[p, q] * lambda_q
    
    psi_dx = np.fft.ifft2(psi_dx_hat)
    psi_dy = np.fft.ifft2(psi_dy_hat)
    
    #psi_dx = np.fft.fftshift(psi_dx)
    #psi_dy = np.fft.fftshift(psi_dy)

    dx = 2*a/X
    dy = 2*a/Y
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dy)
    
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='ij')
    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L

psi_hat = np.fft.fft2(psi)#/N**2
l = L_MK3(psi, psi_hat, a, X, Y)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

Mark IV: Replacing the remaining for loops by numpy magic

In [10]:
@jit
def L_MK4(psi, psi_hat, a, X, Y):
    psi_dx = np.ones((X,Y)) + (0+0j)
    psi_dy = np.ones((X,Y)) + (0+0j)
    psi_dx_hat = np.ones((X,Y)) + (0+0j)
    psi_dy_hat = np.ones((X,Y)) + (0+0j)
    psi_L = np.ones((X,Y)) + (0+0j)
    
    p = np.arange(-X//2, X//2)
    q = np.arange(-Y//2, Y//2)
    
    pp, qq = np.meshgrid(p, q, sparse=False, indexing="ij")
    
    my_p = 2*pp*math.pi/(2*a)
    lambda_q = 2*qq*math.pi/(2*a)
    
    my_p = np.fft.fftshift(my_p)
    lambda_q = np.fft.fftshift(lambda_q)
    
    psi_dx_hat = psi_hat * my_p
    psi_dy_hat = psi_hat * lambda_q
    
    
    psi_dx = np.fft.ifft2(psi_dx_hat)
    psi_dy = np.fft.ifft2(psi_dy_hat)
    

    dx = 2*a/X
    dy = 2*a/Y
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dy)
    
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='ij')
    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L

psi_hat = np.fft.fft2(psi)#/N**2
l = L_MK4(psi, psi_hat, a, X, Y)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

Result: Each one of these functions prduces the same output. Assuming the literal copy of the math provided by the paper is correct, the fast numpy versions of the functions should be correct too!